# Gemini API: Safety Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/blob/main/gemini-guide/quickstarts/Safety_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

The Gemini API has adjustable safety settings. This notebook walks you through how to use them. You'll be able to write a prompt that's blocked, see the reason why, and then adjust the filters to unblock it.

Safety is an important topic, and you can learn much more with the links in the "next steps" section at the end of this notebook. Here, we're focused on the code.

## Prerequisites

You can run this quickstart in [Google Colab](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb), which runs this notebook directly in the browser and does not require additional environment configuration.

Alternatively, to complete this quickstart locally, ensure that your development environment meets the following requirements:

-  Python 3.9+
-  An installation of `jupyter` to run the notebook.

## Setup

### Install the Python SDK

The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:

In [ ]:
!pip install -q -U google-generativeai

### Import packages

Import the necessary packages.

In [ ]:
import google.generativeai as genai

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [ ]:
import os
try:
    from google.colab import userdata
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except ImportError:
    pass

## Prompt Feedback

The result returned by the `Model.generate_content` method is a `genai.GenerateContentResponse` (a wrapper around a `glm.GenerateContentResponse` object).

In [ ]:
model = genai.GenerativeModel('gemini-1.0-pro')

bad_prompt =     # Put your bad prompt here
response = model.generate_content(bad_prompt)

This response object gives you safety feedback in two ways, first the `prompt_feedback` attribute contains a list of safety ratings for the input prompt, and an optional `block_reason`:

In [ ]:
bool(response.prompt_feedback.block_reason)

In [ ]:
response.prompt_feedback

If the prompt is blocked because of the safety ratings, you will not get any candidates in the response:

In [ ]:
response.candidates

### Safety settings

Adjust the safety settings and the prompt is no longer blocked:

In [ ]:
response = model.generate_content(
    bad_prompt,
    safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
    })

With the new settings, the `prompt_feedback` doesn't change, except that the `blocked_reason` is no longer set.

In [ ]:
bool(response.prompt_feedback.block_reason)

And a candidate response is returned.

In [ ]:
len(response.candidates)

You can check `response.text` for the response.

### Candidate ratings

For a prompt that is not blocked, the response object contains a list of `candidate` objects (just 1 for now). Each candidate includes a `finish_reason`:

In [ ]:
candidate = response.candidates[0]
candidate.finish_reason

`FinishReason.STOP` means that the model finished it's output normally. If a candidate's finish reason is `SAFETY` it's becauase the candidate's `safety_ratings` exceeded the the request's `safety_settings` threshold. This response was rated `NEGLIGIBLE` harm probability:

In [ ]:
candidate.safety_ratings

## Learning more

Learn more with these articles on [safety guidance](https://ai.google.dev/docs/safety_guidance) and [safety settings](https://ai.google.dev/docs/safety_setting_gemini) on https://ai.google.dev.
